In [196]:
import importlib
import bike_route_choice

# Make changes to my_module.py

importlib.reload(bike_route_choice)

<module 'bike_route_choice' from 't:\\ABM\\user\\aber\\bike_route_choice\\bike_route_choice.py'>

In [197]:
from bike_route_choice import *

In [3]:
cost_limit_mgra = 4
# zone_level = 'mgra'
cost_limit_taz = 36
# zone_level = 'taz'

In [4]:
random_scale = 0.5

In [5]:
start_time = time.time()
nodes, edges, traversals = read_bike_network_data(zone_level='taz')
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

Reading network data from  T:\ABM\user\aber\bike_route_choice\network
Nodes: (154569, 7) Edges: (366199, 13) Traversals: (724008, 11)
Time taken: 1.22 seconds


In [6]:
num_iterations = 5

In [7]:
# link_num_paths = None

all_paths = []

for i in range(num_iterations):
    # randomize costs
    start_time = time.time()
    edges_rand, traversals_rand = randomize_network_cost(edges, traversals, random_scale)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

    # run dijkstra's
    start_time = time.time()
    centroids, distances, predecessors, node_mapping = perform_dijkstras_algorithm(nodes, edges_rand, limit=cost_limit_taz, zone_level='taz', num_processors=NUM_PRCESSORS)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

    # process paths
    start_time = time.time()
    # all_paths.append(process_paths_new(centroids, predecessors) + (i,))
    paths = process_paths_new(centroids, predecessors)
    all_paths.append(paths + (np.full_like(paths[0],i,dtype=np.uint8),))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")



    # # calculate path size
    # start_time = time.time()
    # print("Calculating path size and creating dataframe...")
    # paths_df =  calculate_path_size(centroids, nodes, edges_rand, node_mapping, link_num_paths, *paths)
    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Time taken: {elapsed_time:.2f} seconds")


Randomizing network costs
Time taken: 0.07 seconds
Need to calculate Dijkstra's on 4947 centroids with 1 processors
Processing Dijkstra's on 4947 centroids with limit=36...
Time taken: 166.20 seconds
Processing paths without numba...
Time taken: 357.32 seconds
Randomizing network costs
Time taken: 0.06 seconds
Need to calculate Dijkstra's on 4947 centroids with 1 processors
Processing Dijkstra's on 4947 centroids with limit=36...
Time taken: 157.02 seconds
Processing paths without numba...
Time taken: 339.78 seconds
Randomizing network costs
Time taken: 0.06 seconds
Need to calculate Dijkstra's on 4947 centroids with 1 processors
Processing Dijkstra's on 4947 centroids with limit=36...
Time taken: 158.10 seconds
Processing paths without numba...
Time taken: 350.21 seconds
Randomizing network costs
Time taken: 0.19 seconds
Need to calculate Dijkstra's on 4947 centroids with 1 processors
Processing Dijkstra's on 4947 centroids with limit=36...
Time taken: 159.59 seconds
Processing paths 

In [10]:
start_time = time.time()
all_paths_concat = map(np.concatenate,zip(*all_paths))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

Time taken: 0.00 seconds


In [11]:
all_paths_orig, all_paths_dest, all_paths_from_node, all_paths_to_node, all_paths_iteration = all_paths_concat

In [198]:
start_time = time.time()
paths_df = calculate_final_logsums(centroids, nodes, edges, node_mapping, all_paths_orig, all_paths_dest, all_paths_from_node, all_paths_to_node, all_paths_iteration, num_iterations)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

t:\ABM\user\aber\bike_route_choice\bike_route_choice.py:476: RuntimeWarning: invalid value encountered in divide
  od_iter_path_size = np.bincount(paths_od_iter_ravel,all_paths_size_component) / od_iter_length_total


Time taken: 3435.62 seconds


In [ ]:
paths_df = paths_df.merge(nodes,how='left',left_on='origin',right_on='id')
paths_df = paths_df.merge(nodes, how='left', left_on = 'destination', right_on = 'id', suffixes = ('_orig','_dest'))

In [ ]:
paths_df[['taz_orig','taz_dest','logsum']].rename(columns={'taz_orig':'i','taz_dest':'j'}).to_csv(r'T:\ABM\user\aber\bike_route_choice\bikeTazLogsum_python_5paths.csv')